In [1]:
import numpy as np
import os
import tensorflow as tf

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

y_test1 = y_test1.astype(np.int32)
###### Do not modify here ###### 
print(X_train1.shape)
print(y_train1.shape)
print(X_valid1.shape)
print(y_valid1.shape)
print(X_test1.shape)
print(y_test1.shape)
train_num = X_train1.shape[0]

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
(28038, 784)
(28038,)
(2558, 784)
(2558,)
(5139, 784)
(5139,)


In [2]:
# claim variables
training_rate = 0.001 #for AdamOptimizer
batch_size = 128
num_epochs = 5
num_display = 50 #for log frequency

num_input = 784
num_classes = 5 # 0~4
dropout = 0.75

# Best validation accuracy seen so far.
best_validation_accuracy = 0.0

# Iteration-number for last improvement to validation accuracy.
last_improvement = 0

# Stop optimization if no improvement found in this many iterations.
require_improvement = 50

# Counter for total number of iterations performed so far.
total_iterations = 0

is_training=True #flag of turn on/off the training precess
model_path_dropout = '/tmp/data/Team06_HW2.ckpt' #path

In [3]:
def layer(layer_name, x, out_channel):
    
    # design the inner structure for each layer
    # He initialization is used
    # ELU activation function is used
    in_channel = x.get_shape()[-1]
    with tf.variable_scope(layer_name):
        w = tf.get_variable(name="weights", 
                            shape=[in_channel, out_channel], 
                            initializer=tf.contrib.layers.variance_scaling_initializer())
        b = tf.get_variable(name="biases",
                            shape=[out_channel],
                            initializer=tf.constant_initializer(0.0))
        x = tf.matmul(x, w, name="matmul") + b
        x = tf.nn.elu(x, name="elu")
        return x

    # 5 hidden layer with 128 neuron each
    # fc layer has 5 channel(0~4)
def Network(x, n_classes, dropout):
    global is_training
    with tf.variable_scope('Network'):
        x = tf.reshape(x, [-1, 784])
        x = layer("layer1", x, 128)
        x = layer("layer2", x, 128)
        x = layer("layer3", x, 128)
        x = layer("layer4", x, 128)
        x = layer("layer5", x, 128)
        x = tf.layers.dropout(x, rate=dropout, training=is_training)
        x = layer("FC", x, 5)    
    return x

In [4]:
# placeholder setting
# Adam optimization is used
# Cross Entropy is used
X = tf.placeholder(tf.float32, [None, num_input], name="Input_X")
Y = tf.placeholder(tf.int64, [None], name="Lable")
is_training= True # training permission accepted

pred = Network(X, num_classes, dropout)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=training_rate)
train_op = optimizer.minimize(loss)

accuracy = tf.equal(tf.argmax(pred, 1), Y)
acc = tf.reduce_mean(tf.cast(accuracy, tf.float32))

saver = tf.train.Saver(tf.global_variables())
summary_op = tf.summary.merge_all()

init = tf.global_variables_initializer()

In [5]:
# feed batch stream instead of directly feed the whole training data
# tf.train.Saver() is used for updating Team06_HW2.ckpt
# early stopping is used
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        is_training=True
        total_cost = 0
        total_batch = int(train_num / batch_size)
        for i in range(total_batch):
            # Increase the total number of iterations performed.
            # It is easier to update it in each iteration because
            # we need this number several times in the following.
            batch_x, batch_y = X_train1[i * batch_size:(i + 1) * batch_size, :], y_train1[i * batch_size:(i + 1) * batch_size]

            # Print status every 100 iterations and after last iteration.
            if (i % num_display == 0) or (i == (total_batch - 1)):
                _, l, ac = sess.run([train_op, loss, acc], feed_dict={X:batch_x, Y:batch_y})

                is_training=False
                val,vaac = sess.run([loss, acc], feed_dict={X:X_valid1, Y:y_valid1})
                print('epoch: ',epoch+1,'step',i+1,' valid_loss:', val,' valid_acc:',vaac)         

                # If validation accuracy is an improvement over best-known.
                if vaac > best_validation_accuracy:
                    # Update the best-known validation accuracy.
                    best_validation_accuracy = vaac

                    # Set the iteration for the last improvement to current.
                    last_improvement = i

                    # Save all variables of the TensorFlow graph to file.
                    save_path = saver.save(sess, model_path_dropout)

            # If no improvement found in the required number of iterations.
            if i - last_improvement > require_improvement:
                print("No improvement found in a while, stopping optimization.")

                # Break out from the for-loop.
                break

epoch:  1 step 1  valid_loss: 1.90806  valid_acc: 0.320172
epoch:  1 step 51  valid_loss: 1.63207  valid_acc: 0.383503
epoch:  1 step 101  valid_loss: 1.42778  valid_acc: 0.463253
epoch:  1 step 151  valid_loss: 1.26626  valid_acc: 0.539484
epoch:  1 step 201  valid_loss: 1.10182  valid_acc: 0.584832
epoch:  1 step 219  valid_loss: 0.997177  valid_acc: 0.643081
epoch:  2 step 1  valid_loss: 0.859295  valid_acc: 0.700547
epoch:  2 step 51  valid_loss: 0.785451  valid_acc: 0.73104
epoch:  2 step 101  valid_loss: 0.718092  valid_acc: 0.745504
epoch:  2 step 151  valid_loss: 0.618108  valid_acc: 0.797889
epoch:  2 step 201  valid_loss: 0.536452  valid_acc: 0.823299
epoch:  2 step 219  valid_loss: 0.500482  valid_acc: 0.833855
epoch:  3 step 1  valid_loss: 0.495135  valid_acc: 0.837764
epoch:  3 step 51  valid_loss: 0.463687  valid_acc: 0.849492
epoch:  3 step 101  valid_loss: 0.457531  valid_acc: 0.845192
No improvement found in a while, stopping optimization.
epoch:  4 step 1  valid_loss:

In [6]:
# handcrafted method for showing recall and precision(test data)
with tf.Session() as sess:
    saver.restore(sess, model_path_dropout)
    is_training=False
    tec,tac,pre = sess.run([loss,acc,pred], feed_dict={X:X_test1, Y:y_test1})
    print('test_loss:', tec,' test_acc:',tac)
    
    pree = np.argmax(pre, 1)
    unique, counts = np.unique(pree, return_counts=True)
    dic_pre = dict(zip(unique, counts))
    
    unique, counts = np.unique(y_test1, return_counts=True)
    dic_real = dict(zip(unique, counts))
    
    correct = 0
    recall = 0
        
    print("Label    Precision       Recall")
    for i in range(5):
        for j in range(pree.shape[0]):
            if pree[j] == i:
                if y_test1[j]==i:
                    correct = correct + 1 
                    
            if y_test1[j] == i:
                if pree[j]==i:
                    recall = recall + 1 
        print(" ",i,"  ",correct/dic_pre[i],"   ",recall/dic_real[i])
        correct = 0
        recall = 0

INFO:tensorflow:Restoring parameters from /tmp/data/Team06_HW2.ckpt
test_loss: 0.332088  test_acc: 0.891613
Label    Precision       Recall
  0    0.903921568627     0.940816326531
  1    0.918005071851     0.956828193833
  2    0.867647058824     0.743217054264
  3    0.825091575092     0.892079207921
  4    0.94375     0.922606924644


In [7]:
# 5-fold
# apply Kfold to get both index of training data and valid data
# Samely,precision and recall of each model is displayed 
from sklearn.model_selection import KFold
fold = KFold(n_splits=5)

sum = 0   
for train_index, valid_index in fold.split(X_train1):
    train_X,valid_X = X_train1[train_index],X_train1[valid_index]
    train_y,valid_y = y_train1[train_index],y_train1[valid_index]
    
    with tf.Session() as sess:

        total_batch = int(train_num*4/ 5 / batch_size)
            
        is_training=True
        sess.run(init)
            
        for i in range(total_batch):
            batch_x, batch_y = train_X[i * batch_size:(i + 1) * batch_size, :], train_y[i * batch_size:(i + 1) * batch_size]
            _, l, ac = sess.run([train_op, loss, acc], feed_dict={X:batch_x, Y:batch_y})
            
        is_training=False

        val,vaac = sess.run([loss, acc], feed_dict={X:valid_X, Y:valid_y})
        sum = sum + vaac
        print('\nvalid_loss:', val,' valid_acc:',vaac)
        #save_path = saver.save(sess, model_path)
        
        tec,tac,pre = sess.run([loss,acc,pred], feed_dict={X:X_test1, Y:y_test1})
        print('test_loss:', tec,' test_acc:',tac)
    
        pree = np.argmax(pre, 1)
        unique, counts = np.unique(pree, return_counts=True)
        dic_pre = dict(zip(unique, counts))
    
        unique, counts = np.unique(y_test1, return_counts=True)
        dic_real = dict(zip(unique, counts))
    
        correct = 0
        recall = 0
        
        print("Label    Precision       Recall")
        for i in range(5):
            for j in range(pree.shape[0]):
                if pree[j] == i:
                    if y_test1[j]==i:
                        correct = correct + 1 
                    
                if y_test1[j] == i:
                    if pree[j]==i:
                        recall = recall + 1 
            print(" ",i,"  ",correct/dic_pre[i],"   ",recall/dic_real[i])
            correct = 0
            recall = 0
            
print("\nFinal average accuracy:",sum/5)


valid_loss: 0.159541  valid_acc: 0.956847
test_loss: 0.123812  test_acc: 0.967309
Label    Precision       Recall
  0    0.943743937924     0.992857142857
  1    0.985952589991     0.989427312775
  2    0.968270214944     0.916666666667
  3    0.959526159921     0.962376237624
  4    0.977528089888     0.974541751527

valid_loss: 0.152418  valid_acc: 0.962019
test_loss: 0.124743  test_acc: 0.967698
Label    Precision       Recall
  0    0.948393378773     0.99387755102
  1    0.983318700615     0.986784140969
  2    0.966565349544     0.924418604651
  3    0.972782258065     0.955445544554
  4    0.965794768612     0.977596741344

valid_loss: 0.16139  valid_acc: 0.958096
test_loss: 0.116773  test_acc: 0.967503
Label    Precision       Recall
  0    0.948493683188     0.995918367347
  1    0.983362521891     0.989427312775
  2    0.965412004069     0.919573643411
  3    0.96890672016     0.956435643564
  4    0.96963562753     0.975560081466

valid_loss: 0.157981  valid_acc: 0.958802
t

In [8]:
#10-fold
from sklearn.model_selection import KFold
fold = KFold(n_splits=10)

sum = 0   
for train_index, valid_index in fold.split(X_train1):
    train_X,valid_X = X_train1[train_index],X_train1[valid_index]
    train_y,valid_y = y_train1[train_index],y_train1[valid_index]
    
    with tf.Session() as sess:

        total_batch = int(train_num*9/ 10 / batch_size)
            
        is_training=True
        sess.run(init)
            
        for i in range(total_batch):
            batch_x, batch_y = train_X[i * batch_size:(i + 1) * batch_size, :], train_y[i * batch_size:(i + 1) * batch_size]
            _, l, ac = sess.run([train_op, loss, acc], feed_dict={X:batch_x, Y:batch_y})
            
        is_training=False

        val,vaac = sess.run([loss, acc], feed_dict={X:valid_X, Y:valid_y})
        sum = sum + vaac
        print('\nvalid_loss:', val,' valid_acc:',vaac)
        #save_path = saver.save(sess, model_path)
        
        tec,tac,pre = sess.run([loss,acc,pred], feed_dict={X:X_test1, Y:y_test1})
        print('test_loss:', tec,' test_acc:',tac)
    
        pree = np.argmax(pre, 1)
        unique, counts = np.unique(pree, return_counts=True)
        dic_pre = dict(zip(unique, counts))
    
        unique, counts = np.unique(y_test1, return_counts=True)
        dic_real = dict(zip(unique, counts))
    
        correct = 0
        recall = 0
        
        print("Label    Precision       Recall")
        for i in range(5):
            for j in range(pree.shape[0]):
                if pree[j] == i:
                    if y_test1[j]==i:
                        correct = correct + 1 
                    
                if y_test1[j] == i:
                    if pree[j]==i:
                        recall = recall + 1 
            print(" ",i,"  ",correct/dic_pre[i],"   ",recall/dic_real[i])
            correct = 0
            recall = 0
            
print("\nFinal average accuracy:",sum/10)


valid_loss: 0.134282  valid_acc: 0.963623
test_loss: 0.0922001  test_acc: 0.972174
Label    Precision       Recall
  0    0.983656792646     0.982653061224
  1    0.981659388646     0.990308370044
  2    0.959244532803     0.93507751938
  3    0.961614173228     0.967326732673
  4    0.973816717019     0.984725050916

valid_loss: 0.149226  valid_acc: 0.958631
test_loss: 0.0924274  test_acc: 0.974119
Label    Precision       Recall
  0    0.972     0.991836734694
  1    0.992939099735     0.991189427313
  2    0.950340798442     0.945736434109
  3    0.976837865055     0.960396039604
  4    0.976673427992     0.980651731161

valid_loss: 0.124668  valid_acc: 0.96398
test_loss: 0.0942674  test_acc: 0.972952
Label    Precision       Recall
  0    0.973     0.992857142857
  1    0.984251968504     0.991189427313
  2    0.960960960961     0.93023255814
  3    0.972111553785     0.966336633663
  4    0.972809667674     0.983706720978

valid_loss: 0.136963  valid_acc: 0.958987
test_loss: 0.09

Here are comment on this tasks:
1. Instead of scan over the data only once,more epoch will get better performance
2. 10-fold seems has better accuracy than 5-fold
3. As we can see, error often occurs in label 2 and 3 than others
4. Drop will boost the accuracy nearly 0.01